[Reference](https://medium.com/@phil.busko/the-welch-cohen-matrix-9e1bd04de98e)

In [2]:
import pandas as PD
import scipy.stats as ST

def get_anova(full_df, group_of, data_col):   return ST.f_oneway( *(full_df.loc[full_df[group_of]==c, data_col] 
                       for c in full_df[group_of].unique()) )

In [5]:
def welch_cohen_matrix(full_df, group_of, data_col):
    
    # get the matrix
    
    group_ls = list(full_df[group_of].unique())
    group_ls.sort()
    welcoh_ls = []
    
    for grp_row in group_ls:
        
        new_dx = {'Sample': grp_row}
        sample_row = full_df.loc[full_df[group_of]==grp_row][data_col]
        
        for grp_col in group_ls:
    
            sample_col = full_df.loc[full_df[group_of]==grp_col][data_col]
        
            welch_tp = ST.ttest_ind(sample_row, sample_col, equal_var=False)
            cohen_nume = abs(sample_row.mean() - sample_col.mean())
            cohen_denom_nume = (sample_row.shape[0]-1) * sample_row.std(ddof=1)**2 + (sample_col.shape[0]-1) * sample_col.std(ddof=1)**2
            cohen_denom_denom = sample_row.shape[0] + sample_col.shape[0] -2
            cohen = cohen_nume / ((cohen_denom_nume/cohen_denom_denom)**(1/2))
            welch = f'{abs(welch_tp[1]): .2f}' if abs(welch_tp[1]) > 0.01 else f'{abs(welch_tp[1]): .0e}'
            wc_value = f'{welch} {cohen : .2f}'
            new_dx[grp_col] = wc_value
        
        welcoh_ls.append(new_dx)
        
    # convert to dataframe for display
    
    welcoh_df = PD.DataFrame(welcoh_ls)
    welcoh_df = welcoh_df[['Sample']+group_ls]
    welcoh_df = welcoh_df.set_index('Sample')
    welcoh_df.style.applymap(get_wc_color)
    return welcoh_df
    
    # function to style the welch-cohen dataframe
    
    def get_wc_color(cell_val):
        cell_ls = cell_val.strip().split()     
        p_value = float(cell_ls[0])
        cohen = float(cell_ls[1])
        color = 'white'
        
        if p_value > 0.05:
            color = 'white'
        elif cohen >= 0.2 and cohen < 0.5:
            color = 'yellow'
        elif cohen >= 0.5 and cohen < 0.8:
            color = 'orange'
        elif cohen >= 0.8:
            color = 'red'
            
        return f'background-color: {color}'